In [1]:
import requests
from bs4 import BeautifulSoup # pip install beautifulsoup4
import pandas as pd
import requests
import re

In [4]:
def get_restaurant_data(start, end):
    restaurant_data = []
    for offset in range(start, end, 30):
        url = f'https://www.tripadvisor.com/RestaurantSearch-g187497-oa{offset}-Barcelona_Catalonia.html#EATERY_LIST_CONTENTS'
        # As tripadvisor recognized python and blocked it I had to use header User-Agent
        html = requests.get(url, headers={'User-Agent': "Mozilla/5.0"})
        soup = BeautifulSoup(html.content, "html.parser")
        
        tags_name = soup.find_all("a", class_="Lwqic Cj b", href=True)
        tags_rating = soup.find_all("svg", class_="UctUV d H0")
        tags_review_count = soup.find_all("span", class_="IiChw")
        tags_cuisine_type = soup.find_all("div", class_="nrKLE PQvPS bAdrM")
        
        names = [i.getText().strip() for i in tags_name]
        # As it is a ranking, the output is (rank. Name of restaurant), so we remove the numbers
        names = [name[name.find(".") + 2:] if name[0].isdigit() else name for name in names]
        ratings = [tag.get('aria-label')[:3] for tag in tags_rating]
        review_counts = [int(tag.getText().strip().split()[0].replace(',','')) for tag in tags_review_count]
        # Some restaurants do not have any cuisine type listed, but if they do have one or multiple, they split on ',' and add everything to a list
        cuisine_types = [[] if tag.find("span", class_="ABgbd") is None else tag.find("span", class_="ABgbd").getText().strip().split(", ") for tag in tags_cuisine_type]
        
        # Zip to add all values in a tuple for each restaurant
        rows = list(zip(names, cuisine_types, ratings, review_counts))
        restaurant_data.extend(rows)
        
    df_ta = pd.DataFrame(restaurant_data, columns=['Name', 'Cuisine_Type', 'Rating', 'Review_Count'])
    return df_ta


In [10]:
df_ta = get_restaurant_data(0, 8520)

In [13]:
df_ta.to_csv(r'../../data/df_ta.csv')

,Name,Cuisine_Type,Rating,Review_Count
0,Bella Venezia,[Italian],5.0,102
1,La Gastronomica Burgers,[Street Food],5.0,1694
2,BelleBuon,"[Italian, Neapolitan]",5.0,3430
3,Desoriente,"[Asian, Japanese Fusion]",5.0,302
4,Vrutal,"[Bar, Healthy]",5.0,1036
...,...,...,...,...
9716,Optimo Bar,"[Bar, Spanish]",5.0,7
9717,Artesanos Espai Gastronomic,"[Italian, Mexican]",4.0,189
9718,Marimorena BCN Tapas,[],5.0,3
9719,Intermezzo,[],5.0,54
